# 第一章 Python数据模型
+ 为什么获取容器大小不使用collection.len()，而是使用len(collection)
+ python是一个框架，数据模型是对框架的描述，规范各种行为
+ 使用框架需要编写方法，供给框架调用

## 1.2 Python风格的纸牌
+ 通过特殊方法利用Python数据模型，类的用户可以不需要记住标准操作的方法的名称，比如size length等
+ 可以充分利用Python标准库，比如choice等
+ 由于getitem的实现，可以使用切片，迭代，反向迭代

In [4]:
import collections
Card = collections.namedtuple('Card',['rank','suit'])
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)]+list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    def __init__(self):
        self._cards = [Card(rank,suit) for suit in self.suits
                      for rank in self.ranks ]
    def __len__(self):
        return len(self._cards)
    def __getitem__(self,position):
        return self._cards[position]

In [5]:
beer_card = Card('7','diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [6]:
deck = FrenchDeck()
len(deck)

52

In [7]:
deck[0]

Card(rank='2', suit='spades')

In [8]:
from random import choice
choice(deck)

Card(rank='9', suit='clubs')

In [9]:
deck[:3] #切片

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [10]:
for d in deck:
    print(d)
    break
for d in reversed(deck):
    print(d)
    break

Card(rank='2', suit='spades')
Card(rank='A', suit='hearts')


## 1.3 特殊方法是如何使用的
+ 特殊方法供Python解释器调用，而不是自己
+ 如果Myobject是用户自定义的类的实例，那么Python将调用你实现的__len__方法
+ 如果是内置类型，list、str、Numpy等，Python会抄近路，会读取PyVarObject中的ob_size的值
+ __init__是个例外，可能会经常调用它来调取超类的初始化方法

In [11]:
import math
class Vector:
    def __init__(self,x=0,y=0):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'Vector({self.x!r},{self.y!r})'#使用!r以标准形式显示属性
    def __abs__(self):
        return math.hypot(self.x,self.y)
    def __bool__(self):
        return bool(abs(self))
    def __add__(self, other):
        x = self.x+other.x
        y = self.y+other.y
        return Vector(x,y)
    def __mul__(self, scalar):
        return Vector(self.x*scalar,self.y*scalar)

V1 = Vector(2,4)
V2 = Vector(2,1)
V1+V2


Vector(4,5)

### 1.3.2 字符串表示形式
+ 特殊方法__repr__供内置函数repr调用，获取对象的字符串表示，如果没有定义，显示为特殊形式
+ 特殊方法__str__供内置函数str()调用，返回终端用户友好的字符串
### 1.3.3 自定义类型的布尔值
+ 默认情况下，用户自定义类型的实例都是真值，除非实现了__bool__或者__len__方法，先以bool()的结果为准，如果没有定义__bool__，那么以len()的结果为准
### 1.3.4 容器API
+ 顶部三个抽象基类Iterable Sized Container只有一个特殊方法，Collection统一了这三个接口
+ Iterable要支持for 拆包和其它迭代方式
+ Sized要支持内置函数len()
+ Container 要支持in运算符
## 1.5 len为什么不是方法

# 第二章 序列
Python中现成可用的序列类型
+ 列表推导式 生成器表达式
+ 元组的两种用法-记录和不可变列表
+ 序列拆包和序列模式
+ 读写切片
+ 专门的序列类型
## 1.内置序列类型
+ 容器序列 list tuple collections.deque 存放不同类型(引用)
+ 扁平序列 str bytes array.array 存放简单相同类型(内存值)
+ 任何Python对象在内存中都有一个包含元数据的标头，例如float
  + ob_refcnt 引用计数
  + ob_type 指向对象类型的指针
  + ob_fval C语言double类型值
## 2.列表推导式和生成器表达式
+ 列表推导式使用filter 和 map两个函数也能有同样的效果
+ 笛卡尔积
+ 生成器表达式占用的内存更小，他是使用迭代器协议组个产出项，而不是构建整个列表供给使用
+ 生成器表达式的句法和列表推导式几乎一样，只不过把方括号改成圆括号
## 3.元组仅仅是不可变列表





In [1]:
#
symbols = '@#$%^&'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
codes

[64, 35, 36, 37, 94, 38]

In [2]:
# 上述代码使用列表推导式
codes = [ord(symbol) for symbol in symbols]
codes

[64, 35, 36, 37, 94, 38]

In [3]:
## 使用map和filter
beyond_ascii = [ord(s) for s in symbols if ord(s) >= 64]
beyond_ascii

[64, 94]

In [5]:
# map和filter并没有更快
beyond_ascii = list(filter(lambda c:c>=64,map(ord,symbols)))
beyond_ascii

[64, 94]

In [6]:
# 生成笛卡尔积 如果写for的话要写两个
colors = ['black','white']
sizes = ["S",'M',"L"]
tshirts = [(c,s) for c in colors for s in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [7]:
# 生成器表达式构建一个元组和数组
tuple(ord(symbol) for symbol in symbols)

(64, 35, 36, 37, 94, 38)

In [8]:
# 第一个参数指定存储类型
import array
array.array('I',(ord(symbol) for symbol in symbols))

array('I', [64, 35, 36, 37, 94, 38])

In [9]:
# 注意生成器表达式的用处
for tshirt in (f'{c},{s}' for c in colors for s in sizes):
    print(tshirt)

black,S
black,M
black,L
white,S
white,M
white,L
